### Video Indexer API Samples (Python)

This notebook provides samples for the following operations in Video Indexer:   

(1) Get account details.   
(2) Upload a video from URL.   
(2A) Upload a video from local file.   
(3) Wait for the video to finish indexing.   
(4) Search for video and get insights.
(5) Use the Widgets API.   
   
Make sure you're logged-in with `az` to authenticate your account.   
   
Copy the `.env.example` file to a new file named `.env`, and update the values with your own account settings.

In [1]:
# imports
from dotenv import dotenv_values
from pprint import pprint
from VideoIndexerClient.Consts import Consts
from VideoIndexerClient.VideoIndexerClient import VideoIndexerClient

Define the following parameters based on your account settings, in case they are different or not defined as environment variables:

In [2]:
config = dotenv_values(".env")

AccountName = config.get('AccountName')
ResourceGroup = config.get('ResourceGroup')
SubscriptionId = config.get('SubscriptionId')

ApiVersion = '2024-01-01'
ApiEndpoint = 'https://api.videoindexer.ai'
AzureResourceManager = 'https://management.azure.com'

# create and validate consts
consts = Consts(ApiVersion, ApiEndpoint, AzureResourceManager, AccountName, ResourceGroup, SubscriptionId)

In [3]:
# Authenticate

# create Video Indexer Client
client = VideoIndexerClient()

# Get access tokens (arm and Video Indexer account)
client.authenticate_async(consts)

#### Sample 1 - Get Account Basic Details
Get account details, not required in most cases.

In [4]:
client.get_account_async()

[Account Details] Id:9a5f0740-eee6-4f8a-8022-f83961dc2f46, Location: japaneast


#### Sample 2 - Index a Video from URL
Upload a video from URL, do not wait for the index operation to complete.   
Define `videoUrl` and `ExcludedAI` first.   
`ExcludedAI` - A list of the AIs you would like to exclude in the format `['Faces', 'Labels', 'Emotions','ObservedPeople']`. Leave empty if you do not want to exclude any AIs. For more details see [here](https://api-portal.videoindexer.ai/api-details#api=Operations&operation=Upload-Video:~:text=AI%20to%20exclude%20when%20indexing%2C%20for%20example%20for%20sensitive%20scenarios.%20Options%20are%3A%20Face/Observed%20peopleEmotions/Labels%7D).

In [5]:
video_url = '<your_video_url>'
video_name = "<your_video_name>"
video_description = "<your_video_description>"
privacy = 'private'
ExcludedAI = []
new_filepath = f"{video_name}_processed.mp4"

cmd = ' '.join([
    '/usr/bin/ffmpeg',
    f'-i {video_url}',
    f'-vf "scale=1280:-2, fps={30}" -vcodec libx265',
    new_filepath,
])

video_id = client.upload_url_async(video_name, video_url, ExcludedAI, False)

Video ID 1mx1et5gjg was uploaded successfully


#### Sample 3 - Polling on Video Completion Event
Wait for the video index to finish (Polling method).

In [6]:
client.wait_for_index_async(video_id)

Checking if video 1mx1et5gjg has finished indexing...
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video index state is Processing
The video 

#### Sample 4 - Get the Video insights
Get the video insights.

In [7]:
insights = client.get_video_async(video_id)
pprint(insights)

Searching videos in account 9a5f0740-eee6-4f8a-8022-f83961dc2f46 for video ID 4ueyn55oe6.
Here are the search results: 
{'partition': None, 'description': None, 'privacyMode': 'Private', 'state': 'Processed', 'accountId': '9a5f0740-eee6-4f8a-8022-f83961dc2f46', 'id': '4ueyn55oe6', 'name': 'meet_short', 'userName': 'Chihiro Kuga', 'created': '2024-12-11T12:10:11.9033333+00:00', 'isOwned': True, 'isEditable': True, 'isBase': True, 'durationInSeconds': 185, 'duration': '0:03:05.166667', 'summarizedInsights': {'name': 'meet_short', 'id': '4ueyn55oe6', 'privacyMode': 'Private', 'duration': {'time': '0:03:05.166667', 'seconds': 185.2}, 'thumbnailVideoId': '4ueyn55oe6', 'thumbnailId': '3e561f09-16ac-4277-a885-9cb083695672', 'faces': [{'videoId': '4ueyn55oe6', 'confidence': 0, 'description': None, 'title': None, 'thumbnailId': '5b648e2f-d0f9-4bf7-b058-715261df7824', 'seenDuration': 185.2, 'seenDurationRatio': 1, 'id': 1156, 'name': 'Unknown #1', 'appearances': [{'startTime': '0:00:00', 'endTim

#### Sample 5 - Widgets API

In [8]:
client.get_insights_widgets_url_async(video_id, widget_type='Keywords')
client.get_player_widget_url_async(video_id)

Getting the insights widget URL for video 4ueyn55oe6
Got the insights widget URL: https://www.videoindexer.ai/embed/insights/9a5f0740-eee6-4f8a-8022-f83961dc2f46/4ueyn55oe6/?location=japaneast&widgets=Keywords&accessToken=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJWZXJzaW9uIjoiMi4wLjAuMCIsIktleVZlcnNpb24iOiI3ZmJkMDkxOGRmMWM0NGNjYTI3ZTA2NGQyYWZkYWViOSIsIkFjY291bnRJZCI6IjlhNWYwNzQwLWVlZTYtNGY4YS04MDIyLWY4Mzk2MWRjMmY0NiIsIkFjY291bnRUeXBlIjoiQXJtIiwiVmlkZW9JZCI6IjR1ZXluNTVvZTYiLCJQZXJtaXNzaW9uIjoiQ29udHJpYnV0b3IiLCJFeHRlcm5hbFVzZXJJZCI6IjkzMjg1NzZCRDcwNjQ5QTk4RDE0MTYwMjUxNjlFNDlFIiwiVXNlclR5cGUiOiJNaWNyb3NvZnRDb3JwQWFkIiwiSXNzdWVyTG9jYXRpb24iOiJqYXBhbmVhc3QiLCJuYmYiOjE3MzM5MTkwODgsImV4cCI6MTczMzkyMjk4OCwiaXNzIjoiaHR0cHM6Ly9hcGkudmlkZW9pbmRleGVyLmFpLyIsImF1ZCI6Imh0dHBzOi8vYXBpLnZpZGVvaW5kZXhlci5haS8ifQ.UzskeSn9P8X6MBlZaadWya3qH0Ph9ICgG8g1K0CD2_-kOlFlsLLZMd-AwNTq1c8slY9dWABfD7hCGt3YwsZF92qe5MRVYePioUg6R4IFg1xMHSOgDe-YETIR-k9HRBtKB-Wz6RWYFOj_FekEZjG8YmyYMjt7lwFHslZPdvybgWYTToA0DasMFeaGfs2zhKpg_0

#### Sample 6 - Prompt Content API

In [9]:
prompt_content = client.get_prompt_content(video_id)
pprint(prompt_content)

Prompt content generation for video_id='4ueyn55oe6' started...
Prompt content is not ready yet. Waiting 5 seconds before checking again...
Prompt content is not ready yet. Waiting 5 seconds before checking again...
Prompt content is not ready yet. Waiting 5 seconds before checking again...
Prompt content is not ready yet. Waiting 5 seconds before checking again...
Prompt content is not ready yet. Waiting 5 seconds before checking again...
{'name': 'meet_short',
 'partition': None,
 'sections': [{'content': '[Video title] meet_short\n'
                          '[Visual labels] electronics, software, multimedia\n'
                          '[OCR] 18:13, '
                          'meet.google.com/iwe-wkws-keg?authuser=0, |□ '
                          'すべてのブックマーク, 大山益弘(画面共有), テキストを入力, '
                          'カスタマー·エクスペリエンス事業本部, DCX, 大山益弘, UX/UI, CMS, 静的サイト, '
                          'モバイルアプリ, '
                          '顧客のCXを高めるために、UX/UIを切り口に、ウェブサイト構築(CMS/静的サイト)、, '
          